<a href="https://colab.research.google.com/github/kalyani179/SMART-WASTE-MANAGEMENT-SYSTEM/blob/main/smart-waste-management-system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
from google.colab import files
import zipfile
import requests

# Step 1: Download the TrashNet Dataset
def download_trashnet_dataset():
    dataset_url = 'https://github.com/garythung/trashnet/archive/refs/heads/master.zip'
    response = requests.get(dataset_url)
    zip_file_path = 'trashnet.zip'

    # Save the zip file
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('.')

    return 'trashnet-master/data/resized_data/dataset-resized'

# Step 2: Load the dataset and prepare data
dataset_path = download_trashnet_dataset()

# Set image size and categories
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
categories = ['plastic', 'paper', 'cardboard', 'trash', 'glass', 'metal']

# Prepare image data for training
def load_images(image_dir):
    images, labels = [], []
    for category in categories:
        category_dir = os.path.join(image_dir, category)
        if not os.path.exists(category_dir):
            print(f"Directory {category_dir} does not exist.")
            continue
        print(f"Loading images from {category_dir}...")
        for img_file in os.listdir(category_dir):
            img_path = os.path.join(category_dir, img_file)
            try:
                img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                images.append(img_array)
                labels.append(categories.index(category))  # Label as index of category
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

# Load the images from the extracted folder
images, labels = load_images(dataset_path)

# Check if images were loaded successfully
if len(images) == 0:
    raise ValueError("No images were loaded. Please check the dataset and directory structure.")

# Step 3: Define and train the CNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(categories), activation='softmax')  # Output layer for categories
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model
model = create_model()
model.fit(images, labels, epochs=10, batch_size=BATCH_SIZE, validation_split=0.2)

# Step 4: Define recycling methods (example data)
recycling_methods = {
    'plastic': {
        'Mechanical Recycling': {'Efficiency': 80, 'Cost': 0.25, 'Energy': 1.5, 'Environmental Impact': 3},
        'Chemical Recycling': {'Efficiency': 85, 'Cost': 0.30, 'Energy': 2, 'Environmental Impact': 4},
    },
    'paper': {
        'Composting': {'Efficiency': 60, 'Cost': 0.05, 'Energy': 0.2, 'Environmental Impact': 2},
    },
    'cardboard': {
        'Recycling': {'Efficiency': 75, 'Cost': 0.10, 'Energy': 0.5, 'Environmental Impact': 3},
    },
    'glass': {
        'Recycling': {'Efficiency': 90, 'Cost': 0.15, 'Energy': 1.0, 'Environmental Impact': 2},
    },
    'metal': {
        'Mechanical Recycling': {'Efficiency': 90, 'Cost': 0.15, 'Energy': 1.5, 'Environmental Impact': 1},
    },
    'trash': {
        'Landfill': {'Efficiency': 50, 'Cost': 0.05, 'Energy': 0.0, 'Environmental Impact': 4},
    },
}

# Step 5: Recommend the best recycling method
def recommend_recycling(waste_type):
    methods = recycling_methods.get(waste_type, {})
    if not methods:
        return "No recycling methods available for this waste type."

    scores = {}
    for method, attributes in methods.items():
        score = (attributes['Efficiency'] * 0.4 +
                 (1/attributes['Cost']) * 0.2 +  # Inverse cost for scoring
                 (1/attributes['Energy']) * 0.2 +  # Inverse energy for scoring
                 attributes['Environmental Impact'] * 0.2)
        scores[method] = score

    best_method = max(scores, key=scores.get)
    return best_method, scores[best_method]

# Example usage
uploaded = files.upload()  # Upload an image for classification
uploaded_image_path = list(uploaded.keys())[0]

# Load the uploaded image and predict waste type
uploaded_img = load_img(uploaded_image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
uploaded_img_array = img_to_array(uploaded_img) / 255.0
uploaded_img_array = np.expand_dims(uploaded_img_array, axis=0)  # Add batch dimension

predicted_class = model.predict(uploaded_img_array)
predicted_label = categories[np.argmax(predicted_class)]

# Recommend recycling method based on predicted waste type
recommended_method, score = recommend_recycling(predicted_label)

print(f"Predicted Waste Type: {predicted_label}")
print(f"Recommended Recycling Method: {recommended_method} (Score: {score})")


Directory trashnet-master/data/resized_data/dataset-resized/plastic does not exist.
Directory trashnet-master/data/resized_data/dataset-resized/paper does not exist.
Directory trashnet-master/data/resized_data/dataset-resized/cardboard does not exist.
Directory trashnet-master/data/resized_data/dataset-resized/trash does not exist.
Directory trashnet-master/data/resized_data/dataset-resized/glass does not exist.
Directory trashnet-master/data/resized_data/dataset-resized/metal does not exist.


ValueError: No images were loaded. Please check the dataset and directory structure.

In [8]:
# Check the contents of the extracted dataset
def check_extracted_contents(directory):
    for root, dirs, files in os.walk(directory):
        print(f"Found directory: {root}")
        for d in dirs:
            print(f"  Subdirectory: {d}")
        for f in files:
            print(f"  File: {f}")

check_extracted_contents('trashnet-master')


Found directory: trashnet-master
  Subdirectory: data
  File: weight-init.lua
  File: LICENSE
  File: train.lua
  File: README.md
  File: DataLoader.lua
  File: utils.lua
  File: plot.lua
  File: test.lua
  File: model.lua
  File: .gitignore
  File: shuffle.lua
Found directory: trashnet-master/data
  File: one-indexed-files-notrash_val.txt
  File: dataset-resized.zip
  File: one-indexed-files-notrash_test.txt
  File: resize.py
  File: one-indexed-files-notrash_train.txt
  File: one-indexed-files.txt
  File: constants.py
  File: zero-indexed-files.txt


In [9]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Define paths and parameters
dataset_path = "trashnet-master/data/resized_data/dataset-resized"
cache_path = "resized_cache"
IMG_HEIGHT, IMG_WIDTH = 256, 256
BATCH_SIZE = 32
EPOCHS = 10
categories = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

# Create cache directory if it doesn't exist
os.makedirs(cache_path, exist_ok=True)

# Load and cache images
def load_and_cache_images(dataset_path, cache_path):
    images, labels = [], []
    for category in categories:
        category_dir = os.path.join(dataset_path, category)
        cache_category_dir = os.path.join(cache_path, category)
        os.makedirs(cache_category_dir, exist_ok=True)

        for img_file in os.listdir(category_dir):
            cache_img_path = os.path.join(cache_category_dir, img_file)
            if os.path.exists(cache_img_path):
                # Load cached image
                img = load_img(cache_img_path)
            else:
                # Load, resize and save to cache
                img_path = os.path.join(category_dir, img_file)
                img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                save_img(cache_img_path, img)

            img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
            images.append(img_array)
            labels.append(categories.index(category))

    return np.array(images), np.array(labels)

# First load and cache images
images, labels = load_and_cache_images(dataset_path, cache_path)

# Validate data loading
if len(images) == 0:
    raise ValueError("No images were loaded. Please check the dataset and directory structure.")
print(f"Loaded {len(images)} images.")

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


FileNotFoundError: [Errno 2] No such file or directory: 'trashnet-master/data/resized_data/dataset-resized/cardboard'

In [5]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(categories), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize model
model = create_model()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the model
model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))


NameError: name 'x_train' is not defined